In [1]:
#Many imports
import retro
import numpy as np
from queue import Queue
import cv2
import itertools
from retro.examples.discretizer import Discretizer
from IPython.display import clear_output, display
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import copy
import time
from matplotlib import pyplot as plt
from operator import itemgetter 
import pickle
import csv

In [2]:
class Env_Wrap:
    def __init__(self, state_len, num_frame_skip):
        #Initiate Environment Wrapper
        self.env = retro.RetroEnv(game='Breakout-Atari2600')
        
        # Define action space in terms of valid combinations of button presses
        combos = [['BUTTON'],['LEFT'],['RIGHT'],[]]
        self.action_space_size = len(combos)
        self.env = Discretizer(self.env, combos=combos)
        
        #Number of frames to skip
        self.num_frame_skip = num_frame_skip 
        self.num_frame_skip_1 = num_frame_skip - 3
        
        #Number of frames in each state
        self.state_len = state_len
        
        _ = self.reset()
        
    def reset(self):
        #Frame buffer
        self.frame_buffer = np.zeros((80,80,2))
        
        #Hold the previous 4 states
        self.state_queue = Queue(maxsize = self.state_len)
        #Fill state queue with empty states
        for i in range(self.state_len):
            self.state_queue.put(np.zeros((80,80)))
        
        self.env.reset()
        #Take NOOP action 
        for _ in range(15):
            state,_,_,_ = self.step(3)
        
        return state
        
    def step(self, action):
        
        next_step, reward, terminal, info = self.frame_skip(action)
        _ = self.state_queue.get()
        self.state_queue.put(next_step)
        state = np.stack(list(self.state_queue.queue)).astype(np.uint8)
        
        return state, reward, terminal, info
    
    def render(self):
        self.env.render()
        
    def close(self):
        self.env.close()
    
    def frame_skip(self, action):
        
        total_reward = 0.0
        for skip in range(self.num_frame_skip):
        
            if skip > self.num_frame_skip_1:
                obs, rew, done, info = self.env.step(action)
                self.frame_buffer[:,:,skip-(self.num_frame_skip-2)] = self.grayscale_downsample_crop(obs)
            
            else:
                obs, rew, done, info = self.env.step(action)
                
            total_reward += rew
        
        max_pool = np.maximum(self.frame_buffer[:,:,0],self.frame_buffer[:,:,1])
        
        return max_pool, total_reward, done, info
    
    def grayscale_downsample_crop(self, frame):
        #Frames are coverted to gray scale, downsampled by 50% and cropped to 80x80 
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        
        (oldh, oldw, oldc) = self.env.observation_space.shape
        ratio = 2
        newshape = (oldw//ratio, oldh//ratio)
        
        downsampled_gray_frame = cv2.resize(gray_frame, newshape, interpolation=cv2.INTER_AREA)
        
        grayscale_downsample_crop_frame = downsampled_gray_frame[20:100,:]
        
        return grayscale_downsample_crop_frame

Run a random agent on the environment.

In [3]:
class Random_Agent():
    def __init__(self):
        pass

    def select_action(self):
        return env_wrap.env.action_space.sample()  # sample random action

random_agent = Random_Agent()

In [4]:
N = 10
max_time_steps = 200
env_wrap = Env_Wrap(4,4)

#run the random agent in the environment for N episodes
for episode in range(N):
    reward_sum = 0
    next_state = env_wrap.env.reset()
    for i in range(max_time_steps):
        action = random_agent.select_action()
        next_state, reward, terminal, info = env_wrap.step(action)
        reward_sum += reward
        #env_wrap.render()
        if terminal:
            clear_output(wait=True)
            time.sleep(0.2)
            print('episode:', episode, 'sum_of_rewards_for_episode:', reward_sum)
            break
print('episode:', episode, 'sum_of_rewards_for_episode:', reward_sum)
#env_wrap.env.render(close=True)

episode: 9 sum_of_rewards_for_episode: 0.0
episode: 9 sum_of_rewards_for_episode: 0.0


Constructing the Q Network

In [5]:
state_dim = 4
action_dim = env_wrap.env.action_space.n

print('Number of observations for a given state:', state_dim)
print('Number of actions for a given state:', action_dim)

Number of observations for a given state: 4
Number of actions for a given state: 4


In [6]:
#Larger network used in nature paper 
class Q_Network(nn.Module):
    def __init__(self, state_dim , action_dim):
        super(Q_Network, self).__init__()
        self.x_layer = nn.Conv2d(state_dim,32,8,stride=4) #convolves 32 filters of 8 x 8 with stride 4
        nn.init.kaiming_normal_(self.x_layer.weight, mode='fan_in', nonlinearity='relu') #He (2015) initialization
        self.h_layer1 = nn.Conv2d(32,64,4,stride=2) #convolves 64 filters of 4 x 4 with stride 2
        nn.init.kaiming_normal_(self.h_layer1.weight, mode='fan_in', nonlinearity='relu')
        self.h_layer2 = nn.Conv2d(64,64,3,stride=1) #convolves 64 filters of 3 x 3 with stride 1
        nn.init.kaiming_normal_(self.h_layer2.weight, mode='fan_in', nonlinearity='relu')
        self.h_layer3 = nn.Linear(6*6*64,512) #512 rectifier units
        nn.init.kaiming_normal_(self.h_layer3.weight, mode='fan_in', nonlinearity='relu')
        self.y_layer = nn.Linear(512, action_dim) #fully-connected linear layer with a single output for each action

    def forward(self, state):
        state = state/255.0
        xh = F.relu(self.x_layer(state))
        hh1 = F.relu(self.h_layer1(xh))
        hh2 = F.relu(self.h_layer2(hh1))
        hh2 = hh2.view(-1,6*6*64)
        hh3 = F.relu(self.h_layer3(hh2))
        state_action_values = self.y_layer(hh3)
        return state_action_values

In [7]:
#Duelling network
#could include different types of advantage value function too
class Duelling_Q_Network(nn.Module):
    def __init__(self, state_dim , action_dim):
        super(Duelling_Q_Network, self).__init__()
        self.x_layer = nn.Conv2d(state_dim,32,8,stride=4) #convolves 32 filters of 8 x 8 with stride 4
        nn.init.kaiming_normal_(self.x_layer.weight, mode='fan_in', nonlinearity='relu') #He (2015) initialization
        
        self.h_layer1 = nn.Conv2d(32,64,4,stride=2) #convolves 64 filters of 4 x 4 with stride 2
        nn.init.kaiming_normal_(self.h_layer1.weight, mode='fan_in', nonlinearity='relu')
        
        self.h_layer2 = nn.Conv2d(64,64,3,stride=1) #convolves 64 filters of 3 x 3 with stride 1
        nn.init.kaiming_normal_(self.h_layer2.weight, mode='fan_in', nonlinearity='relu')
        
        self.advantage = nn.Linear(6*6*64,512) #512 rectifier units
        nn.init.kaiming_normal_(self.advantage.weight, mode='fan_in', nonlinearity='relu')
        
        self.value = nn.Linear(6*6*64,512) #512 rectifier units
        nn.init.kaiming_normal_(self.value.weight, mode='fan_in', nonlinearity='relu')
        
        self.advantage_2 = nn.Linear(512, action_dim)     
        self.value_2 = nn.Linear(512, 1)
        

    def forward(self, state):
        state = state/255.0
        xh = F.relu(self.x_layer(state))
        hh1 = F.relu(self.h_layer1(xh))
        hh2 = F.relu(self.h_layer2(hh1))
        hh2 = hh2.view(-1,6*6*64)
        
        adv = F.relu(self.advantage(hh2))
        val = F.relu(self.value(hh2))
        
        adv = self.advantage_2(adv)
        val = self.value_2(val).expand(hh2.size(0), action_dim)
        
        state_action_values = val + adv - adv.mean(1).unsqueeze(1).expand(hh2.size(0), action_dim)
        return state_action_values

In [8]:
#Smaller network used in Atari DQN paper
'''
class Q_Network(nn.Module):
    def __init__(self, state_dim , action_dim):
        super(Q_Network, self).__init__()
        self.x_layer = nn.Conv2d(state_dim,16,8,stride=4)
        nn.init.kaiming_normal_(self.x_layer.weight, mode='fan_in', nonlinearity='relu')
        self.h_layer1 = nn.Conv2d(16,32,4,stride=2)
        nn.init.kaiming_normal_(self.h_layer1.weight, mode='fan_in', nonlinearity='relu')
        self.h_layer2 = nn.Linear(8*8*32,256)
        nn.init.kaiming_normal_(self.h_layer2.weight, mode='fan_in', nonlinearity='relu')
        self.y_layer = nn.Linear(256, action_dim)

    def forward(self, state):
        xh = F.relu(self.x_layer(state))
        hh1 = F.relu(self.h_layer1(xh))
        hh1 = hh1.view(-1,8*8*32)
        hh2 = F.relu(self.h_layer2(hh1))
        state_action_values = self.y_layer(hh2)
        return state_action_values
'''

"\nclass Q_Network(nn.Module):\n    def __init__(self, state_dim , action_dim):\n        super(Q_Network, self).__init__()\n        self.x_layer = nn.Conv2d(state_dim,16,8,stride=4)\n        nn.init.kaiming_normal_(self.x_layer.weight, mode='fan_in', nonlinearity='relu')\n        self.h_layer1 = nn.Conv2d(16,32,4,stride=2)\n        nn.init.kaiming_normal_(self.h_layer1.weight, mode='fan_in', nonlinearity='relu')\n        self.h_layer2 = nn.Linear(8*8*32,256)\n        nn.init.kaiming_normal_(self.h_layer2.weight, mode='fan_in', nonlinearity='relu')\n        self.y_layer = nn.Linear(256, action_dim)\n\n    def forward(self, state):\n        xh = F.relu(self.x_layer(state))\n        hh1 = F.relu(self.h_layer1(xh))\n        hh1 = hh1.view(-1,8*8*32)\n        hh2 = F.relu(self.h_layer2(hh1))\n        state_action_values = self.y_layer(hh2)\n        return state_action_values\n"

In [9]:
#Test network gives the correct output
qnet = Duelling_Q_Network(state_dim, action_dim)
state = torch.from_numpy(env_wrap.reset()).float().view(-1,4,80,80)
q_values = qnet(state)
print('output of our q_network given a state input: ', q_values)

output of our q_network given a state input:  tensor([[-0.1677, -0.0511,  0.2887,  0.7061]], grad_fn=<SubBackward0>)


# Construct the Replay Buffer

In [10]:
class ReplayBuffer(object):
    def __init__(self, buffer_len):
        self.buffer_len = buffer_len
        self.buffer_states = []
        self.buffer_next_states = []
        self.buffer_actions = []
        self.buffer_rewards = []
        self.buffer_terminals = []
        self.buffer_size = 0
        self.cur_pos = 0

    def add_to_buffer(self, data):
        #data must be of the form (state,next_state,action,reward,terminal)
        
        if self.buffer_size < self.buffer_len:
            self.buffer_size += 1
            self.buffer_states.append(data[0])
            self.buffer_next_states.append(data[1])
            self.buffer_actions.append(data[2]) 
            self.buffer_rewards.append(data[3])
            self.buffer_terminals.append(data[4]) 
        
        else:
            self.buffer_states[self.cur_pos] = data[0]
            self.buffer_next_states[self.cur_pos] = data[1]
            self.buffer_actions[self.cur_pos] = data[2]
            self.buffer_rewards[self.cur_pos] = data[3]
            self.buffer_terminals[self.cur_pos] = data[4]
            self.cur_pos = (self.cur_pos + 1) % self.buffer_len
        
    def sample_minibatch(self,minibatch_length):
        
        if minibatch_length > self.buffer_size:
            #Return entire buffer
            return torch.FloatTensor(self.buffer_states).cuda(), \
                    torch.FloatTensor(self.buffer_next_states).cuda(), \
                    torch.FloatTensor(self.buffer_actions).cuda(), \
                    torch.FloatTensor(self.buffer_rewards).cuda(), \
                    torch.FloatTensor(self.buffer_terminals).cuda()
        
        else:
            
            samples_ind = np.random.randint(0,self.buffer_size,minibatch_length)
        
            return torch.FloatTensor(itemgetter(*samples_ind)(self.buffer_states)).cuda(), \
                    torch.FloatTensor(itemgetter(*samples_ind)(self.buffer_next_states)).cuda(), \
                    torch.FloatTensor(itemgetter(*samples_ind)(self.buffer_actions)).cuda(), \
                    torch.FloatTensor(itemgetter(*samples_ind)(self.buffer_rewards)).cuda(), \
                    torch.FloatTensor(itemgetter(*samples_ind)(self.buffer_terminals)).cuda()

In [11]:
class CUDAReplayBuffer(object):
    def __init__(self, buffer_length):
        self.buffer_length = buffer_length
        gpu = torch.cuda.current_device()
        # Preallocate buffer memory on the GPU
        self.buffer_states = torch.zeros([buffer_length, 4, 80, 80], dtype=torch.uint8, device=gpu)
        self.buffer_next_states = torch.zeros([buffer_length, 4, 80, 80], dtype=torch.uint8, device=gpu)
        self.buffer_actions = torch.zeros([buffer_length, 1], dtype=torch.uint8, device=gpu)
        self.buffer_rewards = torch.zeros([buffer_length, 1], dtype=torch.float32, device=gpu)
        self.buffer_terminals = torch.zeros([buffer_length, 1], dtype=torch.float32, device=gpu)
        self.nb_stored_items = 0
        self.current_position = 0
    def add_to_buffer(self, data):
        # Write at current position
        i = self.current_position
        self.buffer_states[i,:,:,:] = torch.from_numpy(data[0])
        self.buffer_next_states[i,:,:,:] = torch.from_numpy(data[1])
        self.buffer_actions[i, 0] = torch.from_numpy(np.array(data[2], dtype=np.uint8).flatten())
        self.buffer_rewards[i, 0] = torch.from_numpy(np.array(data[3], dtype=np.float32).flatten())
        self.buffer_terminals[i, 0] = torch.from_numpy(np.array(data[4], dtype=np.float32).flatten())
        # If buffer is not already full, increment nb_stored_items
        if self.nb_stored_items < self.buffer_length:
            self.nb_stored_items += 1
        # Increment current write position
        if self.current_position < self.buffer_length - 1:
            self.current_position += 1
        else:
            self.current_position = 0
    def sample_minibatch(self, minibatch_length):
        if minibatch_length > self.nb_stored_items:
            # If we are asked to sample more items than we have stored, just return everything that we
            # have stored instead.
            N = self.nb_stored_items
            return self.buffer_states[:N,:,:,:], self.buffer_next_states[:N,:,:,:], \
                   self.buffer_actions[:N,:], self.buffer_rewards[:N,:], self.buffer_terminals[:N,:]
        else:
            # Otherwise, return a random subset of length `minibatch_length` from the stored items.
            ids = np.random.randint(0, self.nb_stored_items, minibatch_length)
            return self.buffer_states[ids,:,:,:], self.buffer_next_states[ids,:,:,:], \
                   self.buffer_actions[ids,:], self.buffer_rewards[ids,:], self.buffer_terminals[ids,:]

# DQN Agent Implementation

In [12]:
class DQNAgent(object):
    def __init__(self, state_dim, action_dim, gamma, buffer_size, tau, learning_rate, minibatch_size):
        self.qnet = Duelling_Q_Network(state_dim, action_dim)
        self.qnet_target = copy.deepcopy(self.qnet)
        self.discount_factor = gamma
        #self.MSELoss_function = nn.MSELoss()
        self.MSELoss_function = nn.SmoothL1Loss()
        self.replay_buffer = CUDAReplayBuffer(buffer_size)
        self.tau = tau
        self.minibatch_size = minibatch_size
        
        if torch.cuda.is_available():
            self.qnet.cuda()
            self.qnet_target.cuda()
            print('Running model on GPU:',torch.cuda.get_device_name(torch.cuda.current_device()),'!')
        
        self.qnet_optim = torch.optim.Adam( self.qnet.parameters(), lr=learning_rate)
    
    def soft_target_update(self,network,target_network,tau):
        for net_params, target_net_params in zip(network.parameters(), target_network.parameters()):
            target_net_params.data.copy_(net_params.data * tau + target_net_params.data * (1 - tau))
    
    def epsilon_greedy_action(self, state, epsilon):
        if np.random.uniform(0, 1) < epsilon:
            return env_wrap.env.action_space.sample()  # choose random action
        else:
            network_output_to_numpy = self.qnet(state).cpu().data.numpy()
            #network_output = self.qnet(state)
            return np.argmax(network_output_to_numpy)  # choose greedy action
            
    def update_Q_Network(self, state, next_state, action, reward, terminals):
        qsa = torch.gather(self.qnet(state), dim=1, index=action.long())
        qsa_next_action = self.qnet_target(next_state)
        qsa_next_action_max,_ = torch.max(qsa_next_action, dim=1, keepdim=True)
        not_terminals = 1 - terminals
        qsa_next_target = reward + not_terminals * self.discount_factor * qsa_next_action_max
        q_network_loss = self.MSELoss_function(qsa, qsa_next_target.detach())
        self.qnet_optim.zero_grad()
        q_network_loss.backward()
        #Gradient Clipping between (-1, 1)
        for param in self.qnet.parameters():
            param.grad.data.clamp_(-1, 1)
        self.qnet_optim.step()
        
        return q_network_loss
        
    def update(self):
        states, next_states, actions, rewards, terminals = self.replay_buffer.sample_minibatch(self.minibatch_size)
        loss = self.update_Q_Network(states, next_states, actions, rewards, terminals)
        #self.soft_target_update(self.qnet, self.qnet_target, self.tau)
        
        return loss.item()

Train Network

In [13]:
#Learning parameters
total_time_steps = 5000000 #Number of steps to train on
max_time_steps = 10000 #Max number of step in each episode if terminal state not reached
gamma = 0.99 #Discount factor
buffer_size = 100000 #Number of transitions to store in replay memory, 10% total steps
minibatch_size = 32 #Number of samples in minibatch update, 32 used in atari DQN paper
tau = 0.001 #Soft target network update
learning_rate = 0.00025 #Gradient step size
target_update = 10000 #Target network update frequency (number of steps)
learning_starts = 5000 #Model starts learning after x transitions 
update_freq = 4 #Minibatch update frequency in number of steps

#Epsilon decay parameters
epsilon_start = 1.0 #Starting epsilon value
epsilon_mid = 0.1 #Epsion value after some training
epsilon_final = 0.01 #Final epsiolon value
epsilon_decay_length1 = 100000 #total_time_steps//10 #Number of episodes to decay epsilon over, 10% total time steps
epsilon_decay_length2 = total_time_steps - epsilon_decay_length1
eps = epsilon_start

agent = DQNAgent(state_dim, action_dim, gamma, buffer_size, tau, learning_rate, minibatch_size)

#Loop variables
total_steps = 0
episode = 0
save_freq = 100000 #Save checkpoint every x steps
best_reward = -np.inf


#Loss moving average
avg_loss = 0
loss_list_len = 1000
loss_list = [0.0]*loss_list_len
loss_pos = 0

rewards = []
rewards_per_ep = []
losses = []
losses_per_ep = []

while total_steps < total_time_steps:
    episode += 1
    lives = 5
    state = env_wrap.reset()
    clear_output(wait=True)
    print('Current episode:',episode,
          '\nCurrent episode reward:', reward_sum,
          '\nAverage loss:', avg_loss,
          '\nCurrent total steps:',total_steps, 
          '\nGreatest reward:', best_reward,
          '\nCurrent epsilon:', eps)
    reward_sum = 0

    for i in range(max_time_steps):
        
        total_steps += 1
        action = agent.epsilon_greedy_action( torch.from_numpy(state).float().view(-1,4,80,80).cuda() , eps)
        next_state, reward, terminal, info = env_wrap.step(action)
        
        if lives != info['lives']:
            agent.replay_buffer.add_to_buffer( (state,next_state,[action],[reward],[True]) )
            lives = info['lives']
        else:
            agent.replay_buffer.add_to_buffer( (state,next_state,[action],[reward],[terminal]) )
        
        state = next_state
        
        reward_sum += reward
        
        #Update network if learning has started and update frequency reached
        if total_steps > learning_starts and total_steps % update_freq == 0:
            loss = agent.update()
            loss_list[loss_pos] = loss
            loss_pos = (loss_pos + 1)%loss_list_len
            avg_loss = sum(loss_list)/loss_list_len
            losses.append(loss)
        
        #Update target network if target update freqency reached
        if total_steps % target_update == 0:
            agent.qnet_target.load_state_dict(agent.qnet.state_dict())
            
        #Decay epsilon if total steps is less than decay length
        if epsilon_decay_length1 > total_steps:
            eps -= (epsilon_start-epsilon_mid)/epsilon_decay_length1
        else:
            eps -= (epsilon_mid-epsilon_final)/epsilon_decay_length2    
        
        rewards.append(reward)
        
        #Periodically save the model incase crash
        if total_steps % save_freq == 0:
            torch.save({
            'epoch': total_steps,
            'model_state_dict': agent.qnet.state_dict(),
            'optimizer_state_dict': agent.qnet_optim.state_dict()
            }, 'd:/madge/checkpoint_%d.pth'%total_steps)
            
            # write rewards to file 
            with open('d:/madge/rewards', 'a+', newline='') as write_obj:
                csv_writer = csv.writer(write_obj)
                for i in range(len(rewards)):
                    csv_writer.writerow([rewards[i]])
            
            # write losses to file
            with open('d:/madge/losses', 'a+', newline='') as write_obj:
                csv_writer = csv.writer(write_obj)
                for i in range(len(losses)):
                    csv_writer.writerow([losses[i]])

            rewards = []
            losses = []
        
        if terminal:
            #clear_output(wait=True)
            #print('Current total steps:', total_steps, 'Rewards for episode:', reward_sum)
            break
            
    if reward_sum > best_reward:
        best_reward = reward_sum
    
    rewards_per_ep.append(reward_sum)
    losses_per_ep.append(avg_loss)
    

Current episode: 6695 
Current episode reward: 63.0 
Average loss: 0.03007198849786073 
Current total steps: 4999284 
Greatest reward: 306.0 
Current epsilon: 0.010022132653839201


In [14]:
with open('d:/madge/rewards', 'a+', newline='') as write_obj:
    csv_writer = csv.writer(write_obj)
    for i in range(len(rewards)):
        csv_writer.writerow([rewards[i]])
            
# write rewards per episode to file
with open('d:/madge/rewards_per_ep', 'a+', newline='') as write_obj:
    csv_writer = csv.writer(write_obj)
    for i in range(len(rewards_per_ep)):
        csv_writer.writerow([rewards_per_ep[i]])
            
# write losses to file
with open('d:/madge/losses', 'a+', newline='') as write_obj:
    csv_writer = csv.writer(write_obj)
    for i in range(len(losses)):
        csv_writer.writerow([losses[i]])
                    
# write average loss per episode to file
with open('d:/madge/losses_per_ep', 'a+', newline='') as write_obj:
    csv_writer = csv.writer(write_obj)
    for i in range(len(losses_per_ep)):
        csv_writer.writerow([losses_per_ep[i]])

In [15]:
#Save model
torch.save(agent.qnet.state_dict(), 'DQN_model.pth')